In [35]:
def calculate_rs1i(data, column_name='Closing_price', period=14):
    data[column_name] = pd.to_numeric(data[column_name], errors='coerce')
    #data = data.dropna(subset=[column_name])
    
    delta = data[column_name].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(window=period).mean()
    avg_loss = pd.Series(loss).rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi.fillna(0)

def calculate_macd(data: pd.DataFrame, column_name: str = 'Closing_price', short_period: int = 12, long_period: int = 26, signal_period: int = 9) -> pd.DataFrame:
    """
    Calculate the Moving Average Convergence Divergence (MACD) and its Signal Line for a given DataFrame.

    Parameters:
    - data (pd.DataFrame): Input DataFrame containing the price data.
    - column_name (str): The name of the column to calculate MACD on (default is 'Closing_price').
    - short_period (int): The short period for the fast EMA (default is 12).
    - long_period (int): The long period for the slow EMA (default is 26).
    - signal_period (int): The period for the Signal Line (default is 9).

    Returns:
    - pd.DataFrame: A DataFrame containing the MACD, Signal Line, and the MACD Histogram.
    """
    
    # Ensure the column exists
    if column_name not in data.columns:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame.")
    
    # Calculate the short-term EMA (12-period)
    short_ema = data[column_name].ewm(span=short_period, adjust=False).mean()
    
    # Calculate the long-term EMA (26-period)
    long_ema = data[column_name].ewm(span=long_period, adjust=False).mean()
    
    # Calculate the MACD (difference between short-term and long-term EMAs)
    macd = short_ema - long_ema
    
    # Calculate the Signal Line (9-period EMA of the MACD)
    signal_line = macd.ewm(span=signal_period, adjust=False).mean()
    
    # Calculate the MACD Histogram (difference between MACD and Signal Line)
    macd_histogram = macd - signal_line
    
    # Create a DataFrame with the MACD, Signal Line, and Histogram
    macd_df = pd.DataFrame({
        'MACD': macd,
        'Signal_Line': signal_line,
        'MACD_Histogram': macd_histogram
    })
    
    return macd_df

In [38]:
# User selects a currency pair
import joblib
import os
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import swing_technical_indicators 

# Set API Key for Alpha Vantage
#api_key = '89PMD611NPUTS89W'
api_key = 'EWTP9LXAZRDLRHB8'

# Function to fetch Forex data
def fetch_forex_data(from_currency, to_currency):
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'FX_DAILY',
        'from_symbol': from_currency,
        'to_symbol': to_currency,
        'apikey': api_key,
        'outputsize': 'compact'
    }
    response = requests.get(url, params=params)
    data = response.json()

    time_series = data.get('Time Series FX (Daily)', {})
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    df.rename(columns={
        "1. open": "Open_price",
        "2. high": "Day_high",
        "3. low": "Day_low",
        "4. close": "Closing_price"
    }, inplace=True)

    df['Currency Pair'] = f'{from_currency}/{to_currency}'

    # Filter past 60 days to ensure enough data for sequences
    last_60_days = datetime.now() - timedelta(days=60)
    df = df[df.index >= last_60_days]

    return df

# Available currency pairs
currency_pairs = ["USD/INR", "EUR/INR", "JPY/INR", "GBP/INR"]

print("Select a currency pair from the following:")
for i, pair in enumerate(currency_pairs, 1):
    print(f"{i}. {pair}")

choice = int(input("Enter the number corresponding to your choice: "))
selected_pair = currency_pairs[choice - 1]

from_currency, to_currency = selected_pair.split("/")

# Fetch data for selected currency pair
df = fetch_forex_data(from_currency, to_currency)
print(df.columns)
df["Open_price"] = pd.to_numeric(df["Open_price"], errors='coerce')
df["Day_high"] = pd.to_numeric(df["Day_high"], errors='coerce')
df["Day_low"] = pd.to_numeric(df["Day_low"], errors='coerce')
df["Closing_price"] = pd.to_numeric(df["Closing_price"], errors='coerce')

print(df.dtypes)
from statsmodels.tsa.seasonal import seasonal_decompose

def deseasonalize_column(data, column, period=7):
    # Decompose the column
    decomposition = seasonal_decompose(data[column], model='additive', period=period, extrapolate_trend='freq')
    
    # Extract the residual component as deseasonalized data
    deseasonalized = data[column] - decomposition.seasonal
    
    return deseasonalized

# Identify non-stationary columns
non_stationary_columns = ['Day_high', 'Day_low', 'Open_price']

# Create deseasonalized columns
for col in non_stationary_columns:
    deseasonalized_col = deseasonalize_column(df, col, period=14)  # Adjust the period as needed
    df[f'Deseasonalized_{col}'] = deseasonalized_col

df['EMA_100'] = swing_technical_indicators.calculate_100ema(df, 'Closing_price', 100)
df['EMA_200'] = swing_technical_indicators.calculate_200ema(df, 'Closing_price', 200)
df['EMA_50'] = swing_technical_indicators.calculate_100ema(df, 'Closing_price', 50)
df['RSI'] = calculate_rs1i(df,'Closing_price',14)

data_macd = calculate_macd(df, column_name='Closing_price', short_period=12, long_period=26, signal_period=9)
print(data_macd.columns)

for i in data_macd.columns:
    #print(i)
    df[f"{i}"] = data_macd[i]

bollinger_data = swing_technical_indicators.calculate_bollinger_bands(df['Closing_price'])

df = df.join(bollinger_data)
df.drop('Price',axis=1, inplace=True)
df.drop('Closing_price',axis=1, inplace=True)
df.drop('Currency Pair',axis=1, inplace=True)
# Load saved scalers
scaler_X = joblib.load(r'C:\Users\Sejal Hanmante\OneDrive\Documents\GitHub\ForEx\JPYINR\jpyinr_scalerx')
scaler_y = joblib.load(r'C:\Users\Sejal Hanmante\OneDrive\Documents\GitHub\ForEx\JPYINR\jpyinr_scalery')

# Normalize data
df_scaled = scaler_X.transform(df)

# Function to create sequences for CNN
def create_sequences(data, sequence_length=30):
    X, y = [], []
    for i in range(len(data) - sequence_length - 7):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length:i+sequence_length+7, 3])  # Predict next 7 days of Closing Price
    return np.array(X), np.array(y)

# Create sequences
X_seq, _ = create_sequences(df_scaled)

# Ensure correct input shape
sequence_length = 30
feature_count = X_seq.shape[2]  # Number of features

# Define CNN Model
cnn_model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(sequence_length, feature_count)),
    MaxPooling1D(2),
    Dropout(0.2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(7)  # Output layer for predicting next 7 days
])

# Load saved weights
cnn_model.load_weights("cnn_model_weights.h5")

# Predict next 7 days
y_pred_scaled = cnn_model.predict(X_seq[-1].reshape(1, sequence_length, feature_count))

# Convert predictions back to original scale
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Print results
print("\nPredicted Closing Prices for Next 7 Days:")
for i, price in enumerate(y_pred.flatten(), 1):
    print(f"Day {i}: {price:.4f}")

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(range(1, 8), y_pred.flatten(), marker='o', linestyle="--", color="red", label="Predicted Price")
plt.xlabel("Days Ahead")
plt.ylabel("Closing Price")
plt.title(f"Next 7 Days Prediction for {selected_pair}")
plt.legend()
plt.grid()
plt.show()


df.columns


Select a currency pair from the following:
1. USD/INR
2. EUR/INR
3. JPY/INR
4. GBP/INR
Index(['Currency Pair'], dtype='object')


KeyError: 'Open_price'